<a href="https://colab.research.google.com/github/mananmehtagit/ipynb_notebooks/blob/main/LLM_With_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Set up a coding environment locally, make sure that you have a functional Python environment (e.g., Python >3.7) and install the following three Python libraries
Install Streamlit
Install Langchain for Google Gemini AI

In [ ]:
!pip install streamlit langchain_google_genai


#!pip install -q "google-generativeai>=0.7.2"

Get Google API Key

In [5]:
from google.colab import userdata
import google.generativeai as genai
import PIL.Image
from IPython.display import display, Image, HTML
import ipywidgets as widgets

import json
from typing_extensions import TypedDict

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY') #get API key from secret
genai.configure(api_key=GOOGLE_API_KEY)

Write the Integration Code


In [29]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

# Load the Google API key from environment variables
# GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Initialize the Gemini model
llm4 = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key= userdata.get('GOOGLE_API_KEY'))

# Define your query
text = "What is the capital of the USA?"

# Get the response from the model
response = llm4.predict(text)
print(response)

Washington, D.C.


In [ ]:
!fuser -k 5000/tcp #stop all programs running on port 5000 in your Colab environment

In [ ]:
# prompt: modify this, add api routes that takes query and run predict method and return output as reponse, create html page, serve over api that execute this api and display response in return serve traffic over ngrok

from google.colab import userdata
import google.generativeai as genai
import PIL.Image
from IPython.display import display, Image, HTML
import ipywidgets as widgets
import json
from typing_extensions import TypedDict
import os
from langchain_google_genai import ChatGoogleGenerativeAI
# Set up a coding environment locally, make sure that you have a functional Python environment (e.g., Python >3.7) and install the following three Python libraries
# Install Streamlit
# Install Langchain for Google Gemini AI
!pip install streamlit langchain_google_genai pyngrok

#!pip install -q "google-generativeai>=0.7.2"
# Get Google API Key


GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY') #get API key from secret
genai.configure(api_key=GOOGLE_API_KEY)
# Write the Integration Code
#

# Load the Google API key from environment variables
# GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Initialize the Gemini model
llm4 = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key= userdata.get('GOOGLE_API_KEY'))


from flask import Flask, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

@app.route('/predict', methods=['GET'])
def predict():
  query = request.args.get('query')
  if not query:
    return jsonify({'error': 'Query parameter is missing'}), 400
  try:
    response = llm4.predict(query)
    return jsonify({'response': response})
  except Exception as e:
    return jsonify({'error': str(e)}), 500

html_content = """
<!DOCTYPE html>
<html>
<head>
<title>Langchain<>Gemini API</title>
</head>
<body>
<h1>Gemini API</h1>
<form action="/predict" method="get">
  <label for="query">Enter your query:</label><br>
  <input type="text" id="query" name="query"><br><br>
  <input type="submit" value="Submit">
</form>
<div id="response"></div>

<script>
  const form = document.querySelector('form');
  const responseDiv = document.getElementById('response');

  form.addEventListener('submit', async (event) => {
    event.preventDefault();
    const formData = new FormData(form);
    const query = formData.get('query');

    try {
      const response = await fetch(`/predict?query=${query}`);
      const data = await response.json();

      if (data.response) {
        responseDiv.innerHTML = `<p>Response: ${data.response}</p>`;
      } else if (data.error) {
        responseDiv.innerHTML = `<p>Error: ${data.error}</p>`;
      }
    } catch (error) {
      responseDiv.innerHTML = `<p>Error: ${error}</p>`;
    }
  });
</script>
</body>
</html>
"""

@app.route('/')
def index():
  return html_content

if __name__ == '__main__':
  # Start ngrok
  ngrok.set_auth_token(userdata.get('NGROK_AUTH_TOKEN')) # Replace with your ngrok auth token
  public_url = ngrok.connect(5000).public_url
  print(f" * ngrok tunnel \"{public_url}\"")

  # Run the Flask app
  app.run(port=5000)